<a href="https://colab.research.google.com/github/chongzicbo/Dive-into-Deep-Learning-tf.keras/blob/master/5.7.%20%E4%BD%BF%E7%94%A8%E9%87%8D%E5%A4%8D%E5%85%83%E7%B4%A0%E7%9A%84%E7%BD%91%E7%BB%9C(VGG).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##5.7. 使用重复元素的网络（VGG）
AlexNet在LeNet的基础上增加了3个卷积层。但AlexNet作者对它们的卷积窗口、输出通道数和构造顺序均做了大量的调整。虽然AlexNet指明了深度卷积神经网络可以取得出色的结果，但并没有提供简单的规则以指导后来的研究者如何设计新的网络。我们将在本章的后续几节里介绍几种不同的深度网络设计思路。

本节介绍VGG，它的名字来源于论文作者所在的实验室Visual Geometry Group [1]。VGG提出了可以通过重复使用简单的基础块来构建深度模型的思路。

###5.7.1. VGG块
VGG块的组成规律是：连续使用数个相同的填充为1、窗口形状为 $3\times3 $的卷积层后接上一个步幅为2、窗口形状为 $2 \times 2$ 的最大池化层。卷积层保持输入的高和宽不变，而池化层则对其减半。我们使用vgg_block函数来实现这个基础的VGG块，它可以指定卷积层的数量num_convs和输出通道数num_channels。

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Sequential
from tensorflow.data import Dataset
from tensorflow.keras import losses,optimizers
import time
import numpy as np
from tensorflow import image
import os
import sys

In [0]:
tf.enable_eager_execution()

In [0]:
def vgg_block(num_convs,num_channels):
  blk=Sequential()
  for _ in range(num_convs):
    blk.add(layers.Convolution2D(filters=num_channels,kernel_size=3,padding='same',activation='relu'))

  blk.add(layers.MaxPool2D(pool_size=2,strides=2))
  return blk

###5.7.2. VGG网络
与AlexNet和LeNet一样，VGG网络由卷积层模块后接全连接层模块构成。卷积层模块串联数个vgg_block，其超参数由变量conv_arch定义。该变量指定了每个VGG块里卷积层个数和输出通道数。全连接模块则跟AlexNet中的一样。

现在我们构造一个VGG网络。它有5个卷积块，前2块使用单卷积层，而后3块使用双卷积层。第一块的输出通道是64，之后每次对输出通道数翻倍，直到变为512。因为这个网络使用了8个卷积层和3个全连接层，所以经常被称为VGG-11。

In [0]:
conv_arch=((1,64),(1,128),(2,256),(2,512),(2,512))

下面我们实现VGG-11。

In [0]:
def vgg(conv_arch):
  net=Sequential()
  for (num_convs,num_channels) in conv_arch:
    net.add(vgg_block(num_convs,num_channels))
  net.add(layers.Flatten())
  net.add(layers.Dense(4096,activation='relu'))
  net.add(layers.Dropout(0.5))
  net.add(layers.Dense(4096,activation='relu'))
  net.add(layers.Dropout(0.5))
  net.add(layers.Dense(10))
  return net

下面构造一个高和宽均为224的单通道数据样本来观察每一层的输出形状。

In [0]:
net=vgg(conv_arch)

In [0]:
X=tf.random.uniform(shape=(1,224,224,1))
for blk in net.layers:
  X=blk(X)
  print(blk.name,'output shape:\t',X.shape)

sequential_35 output shape:	 (1, 112, 112, 64)
sequential_36 output shape:	 (1, 56, 56, 128)
sequential_37 output shape:	 (1, 28, 28, 256)
sequential_38 output shape:	 (1, 14, 14, 512)
sequential_39 output shape:	 (1, 7, 7, 512)
flatten output shape:	 (1, 25088)
dense_18 output shape:	 (1, 4096)
dropout_10 output shape:	 (1, 4096)
dense_19 output shape:	 (1, 4096)
dropout_11 output shape:	 (1, 4096)
dense_20 output shape:	 (1, 10)


可以看到，每次我们将输入的高和宽减半，直到最终高和宽变成7后传入全连接层。与此同时，输出通道数每次翻倍，直到变成512。因为每个卷积层的窗口大小一样，所以每层的模型参数尺寸和计算复杂度与输入高、输入宽、输入通道数和输出通道数的乘积成正比。VGG这种高和宽减半以及通道翻倍的设计使得多数卷积层都有相同的模型参数尺寸和计算复杂度。

###5.7.3. 获取数据和训练模型
因为VGG-11计算上比AlexNet更加复杂，出于测试的目的我们构造一个通道数更小，或者说更窄的网络在Fashion-MNIST数据集上进行训练。

In [0]:
ratio = 4
small_conv_arch = [(pair[0], pair[1] // ratio) for pair in conv_arch]
net = vgg(small_conv_arch)

In [0]:
lr,num_epochs,batch_size=0.05,5,128
optimizer=optimizers.SGD(learning_rate=lr)
loss=losses.SparseCategoricalCrossentropy(from_logits=True)
buffer_size=1000
def load_data_fashion_mnist(batch_size,buffer_size):
  (x_train,y_train),(x_test,y_test)=keras.datasets.fashion_mnist.load_data()
  x_train=x_train[:,:,:,np.newaxis]#将三维张量增加一个channel维
  x_test=x_test[:,:,:,np.newaxis]
  train_iter=Dataset.from_tensor_slices((x_train,y_train)).map(lambda x,y:(x/255,y)).shuffle(buffer_size).batch(batch_size).map(lambda x,y:(image.resize(images=x,size=(224,224)),y))
  test_iter=Dataset.from_tensor_slices((x_test,y_test)).map(lambda x,y:(x/255,y)).batch(batch_size).map(lambda x,y:(image.resize(images=x,size=(224,224)),y))
  return train_iter,test_iter
train_iter,test_iter=load_data_fashion_mnist(batch_size=batch_size,buffer_size=buffer_size)

, which must contain a single lambda with matching signature. To avoid ambiguity, define each lambda in a separate expression.
, which must contain a single lambda with matching signature. To avoid ambiguity, define each lambda in a separate expression.
, which must contain a single lambda with matching signature. To avoid ambiguity, define each lambda in a separate expression.
, which must contain a single lambda with matching signature. To avoid ambiguity, define each lambda in a separate expression.
, which must contain a single lambda with matching signature. To avoid ambiguity, define each lambda in a separate expression.
, which must contain a single lambda with matching signature. To avoid ambiguity, define each lambda in a separate expression.
, which must contain a single lambda with matching signature. To avoid ambiguity, define each lambda in a separate expression.
, which must contain a single lambda with matching signature. To avoid ambiguity, define each lambda in a separ

In [0]:
net.compile(optimizer=optimizer,loss=loss,metrics=['accuracy'])
history=net.fit_generator(train_iter,validation_data=test_iter,epochs=num_epochs)

Epoch 1/5
 90/469 [====>.........................] - ETA: 1:31:23 - loss: 1.8360 - acc: 0.2180

5.7.4. 小结
* VGG-11通过5个可以重复使用的卷积块来构造网络。根据每块里卷积层个数和输出通道数的不同可以定义出不同的VGG模型。